In [1]:
import os
import json
import re
import nltk
import spacy
import PyPDF2
import pandas as pd

from openai import OpenAI

In [33]:
os.environ['OPENAI_API_KEY'] = ''

In [19]:
client = OpenAI()

In [3]:
# function that extracts text from a PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        print("Number of PDF pages:", len(reader.pages))
        text = ""
        for  page in reader.pages:
            page_text = page.extract_text()
            text += page_text
            #print(page_text)
    return text

In [17]:
# Vectorize the query using OpenAI's text-embedding-ada-002 model
def get_embedding(text):
    # vectorize with OpenAI text-emebdding-ada-002
    embedding = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002")

    vector = embedding.data[0].embedding
    
    return vector

# Chunking

In [4]:
def split_sentences(text):
    
    sentences = re.split('[.|?]', text)
    sentences = [sentence.strip() for sentence in sentences if sentence.split()]

    return sentences

def split_sentences_by_nltk(text):

    chunks = []
    for sentence in nltk.sent_tokenize(text):
        chunks.append(sentence)
    
    return chunks

def split_sentences_by_spacy(text, max_tokens, overlap=0, model='en_core_web_sm'):

    nlp = spacy.load(model)
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    return sentences

    tokens_lengths = [count_tokens(sent) for sent in sentences]

    chunks = []
    start_idx = 0

    while start_idx < len(sentences):

        current_chunk = []
        current_token_count = 0
        for idx in range(start_idx, len(sentences)):
            if current_token_count + tokens_lengths[idx] > max_tokens:
                break

            current_chunk.append(sentences[idx])
            current_token_count += tokens_lengths[idx]

        chunks.append(' '.join(current_chunk))

        if overlap >= len(current_chunk):
            start_idx += 1
        else:
            start_idx += len(current_chunk) - overlap


    return chunks

In [5]:
texto = """Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
           Entry requirements for China are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
              - Visa.
              - Proof of accommodation."""

In [6]:
s_textos = split_sentences_by_nltk(texto)

for i in range(len(s_textos)):
    print(i, s_textos[i])

0 Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
1 - Return ticket.
2 Entry requirements for China are as follows:
              - Passport with validity of at least six months.
3 - Return ticket.
4 - Visa.
5 - Proof of accommodation.


In [7]:
s_textos = split_sentences_by_spacy(texto, max_tokens=20)
for i in range(0, len(s_textos)):
   print(i, s_textos[i])

0 Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
           
1 Entry requirements for China are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
              - Visa.
              
2 - Proof of accommodation.


# PDF

In [9]:
texto_pdf = extract_text_from_pdf('/home/armando/git/gen_ai/generative/condiciones_generales.pdf')
print(len(texto_pdf))

Number of PDF pages: 38
186037


In [14]:
#sub_texto = texto_pdf.split('CAPITULO I – Generalidades')[2]
#sub_texto = sub_texto.split('CAPITULO II')[1]
#sub_texto = sub_texto.split('CAPITULO III')[0]

In [8]:
array_tag = ['asistencia medica', 'asistencia medica', 'traslados', 'repatriacion', 'traslados', 'reintegro',
             'soporte', 'soporte', 'soporte', 'repatriacion', 'equipaje', 'reintegro', 'soporte', 'soporte', 
             'deportes' , 'deportes']
array_titulo = ['Asistencia Médica por enfermedad o accidente',
                'Prótesis y ortesis',
                'Traslados sanitarios', 
                'Repatriaciones Sanitarias',
                'Traslado de familiar',
                'Reintegro de gastos por imposibilidad de embarque',
                'Transmisión de mensajes urgentes', 
                'Asistencia en caso de extravío de documentos y/o tarjetas de crédito', 
                'Línea de Consultas', 
                'Repatriación o Traslado de restos', 
                'Compensación por daños en el equipaje (tramos marítimos/fluviales/aéreos)', 
                'Regreso anticipado por siniestro en el domicilio', 
                'Asistencia Legal', 
                'Transferencia de fondos:', 
                'Práctica recreativa de deportes', 
                'Upgrade Deporte']
array_contenido = ["""Los servicios de asistencia médica ofrecidos por la prestadora comprenden exclusivamente el tratamiento de 
urgencia del cuadro agudo que impida la continuación del viaje, e incluyen: 
a) Consultas médicas.  La prestadora  podrá coordinar visitas a domicilio siempre que: i) el tipo de 
emergencia lo amerite, ii) haya disponibilidad en la red de proveedores de la prestadora  de médicos que 
brinden estos servicios en la localidad donde deba asistirse la emergencia; iii) los médicos tengan 
disponibilidad en el horario en que sea requerido.  
b) Teleasistencia médica: Según la naturaleza de la asistencia, la prestadora  podrá coordinar y gestionar 
esta prestación cuyo objetivo es orientar y proporcionar información general sobre salud. Dependiendo del 
país en que se encuentre el pasajero el servicio podrá incluir la emisión de una receta médica y/o 
medicamentos siempre vinculados con el diagnóstico motivo de la consulta. Incluso en las situaciones en 
las que se pueda emitir una receta médica existirán limitaciones sobre todos aquellos medicamentos 
controlados por las leyes locales y/o que contengan psicofármacos, en cuyos casos solo podrá emitirse una 
receta médica por medio de una asistencia médica presencial. 
c) Atención por especialista: cuando sea indicada por los equipos médicos de urgencia y autorizada 
previamente por la Central Operativa de la prestadora .""", 
                  """la prestadora tomará a cargo los gastos de prótesis y ortesis, únicamente cuando esté 
indicado en el cuadro de prestaciones del voucher y hasta el sublímite monetario allí indicado. Se obviará, en 
consecuencia, la exclusión 50)xxvii). Los gastos por esta prestación serán imputados a cuenta del Límite de 
Gastos de Asistencia Médica indicado en el cuadro de prestaciones.""",
                  """en casos de emergencia, la prestadora organizará el traslado al centro asistencial más 
próximo para que el Titular herido o enfermo reciba atención médica. En caso de que el médico tratante 
aconseje el traslado a otro centro asistencial más adecuado, previa autorización del Departamento Médico de 
la prestadora , se procederá a la organización del mismo, según las posibilidades del caso, en las condiciones 
y medios autorizados por la Central Operativa interviniente.  
Únicamente razones de índole médica, evaluadas a criterio exclusivo del Departamento Médico de la 
prestadora , serán tenidas en cuenta para decidir la procedencia y/o urgencia del traslado del Titular. Siempre 
que el médico tratante aconsejara el traslado sanitario del Titular, éste deberá ser médica y científicamente 
justificado por escrito.""", 
                  """cuando el Departamento Médico de la prestadora estime necesario efectuar la 
repatriación sanitaria del Titular, como consecuencia de enfermedad o accidente grave, la repatriación del 
herido o enfermo será efectuada, previa autorización del Departamento Médico de la prestadora , según las 
posibilidades del caso, en las condiciones y medios autorizados por la Central Operativa interviniente, hasta el 
país del domicilio real del Titular. Esta repatriación asimismo deberá ser autorizada y justificada médica y 
científicamente por el médico tratante. Si el Titular o sus acompañantes decidieran efectuar la repatriación 
dejando de lado la opinión del Departamento Médico de la prestadora , ninguna responsabilidad recaerá sobre 
la prestadora por dicha actitud, siendo la repatriación, sus costos y consecuencias por cuenta y riesgo del 
Titular y/o sus acompañantes, sin derecho a repetición contra la prestadora .  
Los gastos por esta prestación serán imputados a cuenta del Límite de Gastos de Asistencia Médica indicado 
en el cuadro de prestaciones. """, 
                  """  
a) en caso de aislamiento social médico prescripto por profesional médico de aquel Titular con diagnóstico 
positivo COVID-19 producida por el virus SARS-COV2 habiendo sido la asistencia coordinada por la 
prestadora , , la prestadora se hará cargo de un billete aéreo ida y vuelta en clase económica desde el 
país de residencia del Titular, para que un familiar del Titular lo acompañe, siempre que el mismo hubiera 
viajado sin acompañantes (o con menores) y se encontrara solo en destino al tiempo de la hospitalización. 
Los billetes aéreos estarán sujetos a disponibilidad de lugar. Los gastos por esta prestación serán 
imputados a cuenta del Límite de Gastos de Asistencia Médica indicado en el cuadro de prestaciones . 
b) en caso de que la hospitalización de un Titular, autorizada por la prestadora , sea superior a 5 (cinco) días 
corridos, la prestadora se hará cargo de un billete aéreo ida y vuelta en clase económica desde el país de 
residencia del Titular, para que un familiar del Titular lo acompañe, siempre que el mismo hubiera viajado 
sin acompañantes (o con menores) y se encontrara solo en destino al tiempo de la hospitalización. Los 
billetes aéreos estarán sujetos a disponibilidad de lugar.  Este beneficio se otorgará únicamente en la 
medida que los primeros 5 (cinco) días de hospitalización estén comprendidos dentro de la vigencia del 
voucher del Titular. Los gastos por esta prestación serán imputados a cuenta del Límite de Gastos de 
Asistencia Médica indicado en el cuadro de prestaciones .""", """La prestadora  reintegrará los gastos de hotel (sin extras, es decir solamente alojamiento) del Titular que, por 
razones médicas comprobables por el Departamento Médico de la prestadora , se vea imposibilitado de 
embarcar en el vuelo de regreso a su país de residencia. Esta prestación se brindará desde la fecha original 
de embarque y tendrá un límite monetario indicado en el cuadro de prestaciones. Los gastos por esta 
prestación serán imputados a cuenta del Límite de Gastos de Asistencia Médica indicado en el cuadro de 
prestaciones .""",
                  """
la prestadora se encargará de transmitir los mensajes urgentes y 
justificados de los Beneficiarios  relativos a cualquiera de los eventos objeto de las prestaciones de esta 
modalidad.""", """ 
la prestadora asesorará al Titular 
en el caso que hubiera extraviado sus documentos y/o tarjetas de crédito.""",
                   """la prestadora brindará información al Titular,  a través de su Central Operativa y durante 
las 24 horas, los 365 días del año, referente a:  
a) Hoteles, clima, moneda, consulados, embajadas, etc. de su lugar de destino.  
b) Consejos generales previos al viaje en relación a las patologías y/o necesidades del cliente. 
c) Información sobre medicamentos equivalentes en caso de extravío u olvido de su medicina habitual.""",""" 
 en caso de fallecimiento del Titular de un voucher durante el viaje, la 
prestadora organizará y tomará a su cargo, por el medio que considere más conveniente, los gastos de féretro  
 de traslado o cremación en el lugar del hecho, trámites administrativos (en lo que esté a su alcance) y transporte 
hasta: 
a) Para aquellos vouchers  que confieren derecho a asistencia en el ámbito internacional: el lugar de ingreso 
al país del domicilio real del Titular. Los traslados dentro del país de residencia del Titular no estarán a 
cargo de la prestadora ; 
b) Para aquellos vouchers  que confieren derecho a asistencia en el ámbito nacional: el lugar de inhumación 
que determinen sus familiares directos, en la ciudad del domicilio real del Titular. 
c) En caso de deceso del Titular del voucher , y si éste hubiera viajado sin acompañantes y se encontrara solo 
(o con menores) en destino, la prestadora  abonará los gastos de traslado y alojamiento de un familiar 
(desde el lugar de residencia del titular) para que pueda realizar los trámites y gestiones de traslado 
necesarias. Los tickets serán en clase económica y estarán sujetos a disponibilidad de la compañía de 
transporte de pasajeros. Por concepto de hotel (alojamiento, sin extras), la prestadora  asumirá hasta USD 
150 (dólares estadounidenses ciento cincuenta) por noche, por un máximo de tres noches.  
Los gastos de féretro definitivo, trámites funerarios e inhumación no estarán en ningún caso a cargo de la 
prestadora . """, """ 
Si alguno de los bultos del equipaje despachado en bodega por el Titular sufriera daños tales que impidan su 
normal uso, la prestadora abonará al Titular una suma en concepto de indemnización por daños en el equipaje, 
cuyo límite se indica en el cuadro de prestaciones . 
Quedan excluidos daños estéticos, aquellos que no afecten la funcionalidad del bien y los derivados del normal 
desgaste. 
Para la procedencia de este beneficio debe verificarse: 
a) que el voucher del Titular incluya este beneficio para el medio de transporte que da lugar al reclamo. 
b) que la rotura haya ocurrido entre el momento en que el equipaje fue embarcado y el momento en que deba 
ser entregado al pasajero al desembarcar;  
c) el Titular deberá realizar la denuncia pertinente ante la empresa transportista, y presentar la misma a la 
prestadora;   
d) el Titular deberá notificar a la prestadora  dentro de las 48 horas de emitido el formulario de denuncia ante 
la compañía transportista.""", """en caso de ocurrencia de robo con violencia de puertas o 
ventanas, incendio o explosión en el domicilio real del Titular, con riesgo a producirse mayores daños, la 
prestadora  tomará a cargo los gastos de regreso correspondientes del Titular del voucher  hasta el domicilio 
siniestrado. Si la vivienda perteneciera a dos o más Titulares de vouchers , la prestadora tomará a su cargo 
solamente el regreso de uno solo de los Titulares. Se entiende por gastos de regreso a la penalidad por cambio 
de fecha del pasaje de regreso o la diferencia de costo de un pasaje de vuelta desde el domicilio donde se 
encuentre el Titular del voucher  hasta el domicilio siniestrado. En los casos en los que el pasajero hubiera 
adquirido un ticket que no admite cambios ni siquiera abonando una penalidad, entonces la prestadora  
reintegrará un monto máximo de USD 200 (doscientos dólares estadounidenses)""", """la prestadora pondrá a disposición del Titular que lo solicite un abogado que se haga cargo 
de su defensa civil o criminal con motivo de imputársele responsabilidad por un accidente de tránsito. La 
prestadora se hará cargo de un importe destinado a cubrir el pago de los honorarios de dicho abogado hasta 
el límite máximo que se detalla en el cuadro de prestaciones . La asistencia no se brindará ni se reconocerá 
importe alguno en caso de que el Titular utilice un profesional que no fuera designado o autorizado 
fehacientemente por la prestadora . """, """en caso de emergencia justificada y a criterio de la prestadora , el Titular podrá 
solicitar la transferencia de una suma de dinero acorde a la emergencia acontecida y hasta el límite máximo 
que por viaje corresponda al tipo de voucher  contratado y que surge de lo informado en el cuadro de 
prestaciones . El costo de la transferencia de fondos estará a cargo de la prestadora . Se entiende que el 
Titular deberá arbitrar los medios para que la prestadora  reciba antes de efectuar la transferencia la totalidad 
del monto a transferir en sus oficinas o donde le sea solicitado. La prestadora  no gestionará la compra o venta 
de moneda extranjera ni será responsable en aquellos casos en los que las disposiciones legales y/o 
impositivas de los países involucrados en el envío de fondos restrinjan o impongan condiciones diferentes a las 
requeridas por el titular del voucher . """, """ 
a) Aplicación : la prestadora cubrirá las asistencias médicas derivadas de la práctica recreativa de deportes 
hasta el monto indicado en el cuadro de prestaciones del voucher siempre que las mismas: 
i) Tengan carácter recreativo, es decir que no son el “motivo” del viaje del titular, sino una actividad que 
surge de manera espontánea y ocasional, 
ii) Se lleven a cabo en ámbitos adecuados y con las medidas de seguridad necesarias, 
b) Exclusiones particulares: 
i) Asistencias derivadas de la participación del titular en competencias deportivas sean amateurs o 
profesionales. 
ii) Asistencias derivadas de la participación del titular en prácticas, entrenamientos, etc. de índole amateur 
o profesional. 
iii) Asistencias derivadas de la participación en deportes peligrosos.""", """exclusivamente  para aquellos vouchers  con el upgrade de DEPORTES  y dados los 
supuestos indicados en las presentes Condiciones Generales,  la prestadora brindará adicionalmente al Titular 
los siguientes servicios:  
a) Gastos por búsqueda y salvamento: la prestadora tomará a su cargo, hasta el límite máximo de USD 
5.000 (dólares estadounidenses cinco mil) por los gastos originados para la búsqueda y salvamento al 
momento de organizar el rescate del Titular, ocasionados por la práctica deportiva de carácter amateur, 
exclusivamente si el lugar en que se produce el evento no cuenta con los medios necesarios a tal fin. Los 
gastos por esta prestación serán imputados a cuenta del Límite de Gastos de Asistencia Médica indicado 
en la cláusula 9)m). 
b) La prestadora  brindará a su cargo los servicios de Asistencia Médica requeridos como consecuencia de la 
práctica o la participación del Titular en competencias de cualquier deporte (sean amateurs o profesionales), 
siempre y cuando las mismas se realicen en los ámbitos expresamente adecuados y/o autorizados a tal fin; 
se obviarán en consecuencia y al efecto de esta prestación las exclusiones mencionadas en la cláusula 
50)xiv) y 50)xv). Los gastos por esta prestación serán imputados a cuenta del Límite de Gastos de 
Asistencia Médica indicado en la cláusula 9)m). 
c) Exclusiones Particulares: se encuentran excluidos de la prestación asistencial, tanto para los supuestos 
de enfermedad como de Accidente, los tratamientos de los eventos que se detallan a continuación: 
i) Consecuencias derivadas de la práctica, tanto profesional como amateur de paracaidismo, salto base, 
wingsuit, montañismo a más de 3000 msnm, escalada en roca, buceo, boxeo, motociclismo, 
automovilismo, descenso ciclismo de montaña, bungee-jumping y los denominados deportes extremos 
en general. 
d) El resto de  las Condiciones Generales mantiene plena vigencia, salvo aquellos aspectos expresamente 
modificados por la presente cláusula. """]

In [9]:
print(len(array_tag))
print(len(array_titulo)) 
print(len(array_contenido))

16
16
16


In [10]:
i = 15
print(array_tag[i])
print(array_titulo[i])
print(array_contenido[i])

deportes
Upgrade Deporte
exclusivamente  para aquellos vouchers  con el upgrade de DEPORTES  y dados los 
supuestos indicados en las presentes Condiciones Generales,  la prestadora brindará adicionalmente al Titular 
los siguientes servicios:  
a) Gastos por búsqueda y salvamento: la prestadora tomará a su cargo, hasta el límite máximo de USD 
5.000 (dólares estadounidenses cinco mil) por los gastos originados para la búsqueda y salvamento al 
momento de organizar el rescate del Titular, ocasionados por la práctica deportiva de carácter amateur, 
exclusivamente si el lugar en que se produce el evento no cuenta con los medios necesarios a tal fin. Los 
gastos por esta prestación serán imputados a cuenta del Límite de Gastos de Asistencia Médica indicado 
en la cláusula 9)m). 
b) La prestadora  brindará a su cargo los servicios de Asistencia Médica requeridos como consecuencia de la 
práctica o la participación del Titular en competencias de cualquier deporte (sean amateurs o profesional

In [39]:
#GENERAR EMBEDDINGS PARA CADA CHUNK
array_vectores = []
array_textos = []
for i in range(0, len(array_contenido)):

    chunks = split_sentences_by_spacy(array_contenido[i], max_tokens=3000, overlap=10)
    print('Chunks for term ' + str(i) + ' are ' + str(len(chunks)))
    array_sub_vectores = []
    array_sub_texto = []

    for j in range(0, len(chunks)):
        texto_procesado = chunks[j].replace('\n', ' ')
        vector = get_embedding(texto_procesado)
        array_sub_texto.append(texto_procesado)
        array_sub_vectores.append(vector)

        #break

    array_textos.append(array_sub_texto)
    array_vectores.append(array_sub_vectores)
    #break
    

Chunks for term 0 are 7
Chunks for term 1 are 4
Chunks for term 2 are 6
Chunks for term 3 are 4
Chunks for term 4 are 9
Chunks for term 5 are 3
Chunks for term 6 are 1
Chunks for term 7 are 1
Chunks for term 8 are 4
Chunks for term 9 are 9
Chunks for term 10 are 8
Chunks for term 11 are 6
Chunks for term 12 are 6
Chunks for term 13 are 7
Chunks for term 14 are 8
Chunks for term 15 are 14


In [40]:
print(len(array_vectores))
print(len(array_textos))

16
16


In [45]:
output_df = pd.DataFrame({'tag': array_tag, 'titulo': array_titulo, 'descripcion': array_contenido, 
                          'contenido': array_textos, 'embedding': array_vectores})
print(output_df.shape)
display(output_df.head(3))

(16, 5)


,tag,titulo,descripcion,contenido,embedding
0,asistencia medica,Asistencia Médica por enfermedad o accidente,Los servicios de asistencia médica ofrecidos p...,[Los servicios de asistencia médica ofrecidos ...,"[[-0.019654173403978348, -0.009384604170918465..."
1,asistencia medica,Prótesis y ortesis,la prestadora tomará a cargo los gastos de pró...,[la prestadora tomará a cargo los gastos de pr...,"[[0.0008464913116768003, -0.009172453545033932..."
2,traslados,Traslados sanitarios,"en casos de emergencia, la prestadora organiza...","[en casos de emergencia, la prestadora organiz...","[[-0.009860559366643429, -0.013706246390938759..."


In [46]:
output_json = output_df.to_dict(orient='index')
print(output_json[0]['descripcion'])

Los servicios de asistencia médica ofrecidos por la prestadora comprenden exclusivamente el tratamiento de 
urgencia del cuadro agudo que impida la continuación del viaje, e incluyen: 
a) Consultas médicas.  La prestadora  podrá coordinar visitas a domicilio siempre que: i) el tipo de 
emergencia lo amerite, ii) haya disponibilidad en la red de proveedores de la prestadora  de médicos que 
brinden estos servicios en la localidad donde deba asistirse la emergencia; iii) los médicos tengan 
disponibilidad en el horario en que sea requerido.  
b) Teleasistencia médica: Según la naturaleza de la asistencia, la prestadora  podrá coordinar y gestionar 
esta prestación cuyo objetivo es orientar y proporcionar información general sobre salud. Dependiendo del 
país en que se encuentre el pasajero el servicio podrá incluir la emisión de una receta médica y/o 
medicamentos siempre vinculados con el diagnóstico motivo de la consulta. Incluso en las situaciones en 
las que se pueda emitir una recet

In [47]:
with open('terminos.json', 'w') as f:
    json.dump(output_json, f)